# FL Client over Secure RPC

In this notebook, we will present how to launch a gRPC client as an FL client with an authenticator. To pair with the server notebook, we consider only one client.

In [1]:
num_clients = 1

## Load client configurations

We load the configuration for the client from `examples/configs/mnist/client_1.yaml`

In [2]:
from omegaconf import OmegaConf
client_config_file = "../../examples/configs/mnist/client_1.yaml"
client_config = OmegaConf.load(client_config_file)
print(OmegaConf.to_yaml(client_config))

train_configs:
  device: cpu
  logging_id: Client1
  logging_output_dirname: ./output
  logging_output_filename: result
data_configs:
  dataset_path: ./dataset/mnist_dataset.py
  dataset_name: get_mnist
  dataset_kwargs:
    num_clients: 2
    client_id: 0
    partition_strategy: class_noniid
    visualization: true
    output_dirname: ./output
    output_filename: visualization.pdf
comm_configs:
  grpc_configs:
    server_uri: localhost:50051
    max_message_size: 1048576
    use_ssl: false



💡 We need to change the relative path in `data_configs.dataset_path` to point to the right file relative to this notebook.

💡 We also need to change `data_configs.dataset_kwargs.num_clients` to 1 to make sure we only partition the MNIST dataset to one client split. We change `data_configs.dataset_kwargs.visualizaton` to False as well.

In [3]:
client_config.data_configs.dataset_path = '../../examples/dataset/mnist_dataset.py'
client_config.data_configs.dataset_kwargs.num_clients = num_clients
client_config.data_configs.dataset_kwargs.visualization = False

## Create secure SSL channel and authenticator

The client requires a root certificate to verify the server certificate. In this example, we provide that [root certificate](../../src/appfl/comm/grpc/credentials/root.crt), assuming that the server uses self-signed [certificate](../../src/appfl/comm/grpc/credentials/localhost.crt) and [key](../../src/appfl/comm/grpc/credentials/localhost.key) provided by gRPC official documentation.

To use the provided root certificate, user just to need to set the following. If the user would like to use his own root certificate, just change this to the file path to the local root certificate.

In [4]:
client_config.comm_configs.grpc_configs.use_ssl = True
client_config.comm_configs.grpc_configs.root_certificate = "../../src/appfl/comm/grpc/credentials/root.crt"

We also need to set configurations to use the naive authenticator and provide the `auth_token` agreed with the server for authentication.

In [5]:
client_config.comm_configs.grpc_configs.use_authenticator = True
client_config.comm_configs.grpc_configs.authenticator = "NaiveAuthenticator"
client_config.comm_configs.grpc_configs.authenticator_args = {"auth_token": "A_SECRET_DEMO_TOKEN"}

## Create the client agent and communicator

Now we are ready to create the client agent using the `client_agent` defined and modified abouve, as well as a `GRPCClientCommunicator` to send request to the server.

⚠️ Please make sure that you have started the server from the other notebook!

In [6]:
from appfl.agent import ClientAgent
from appfl.communicator.grpc import GRPCClientCommunicator
client_agent = ClientAgent(client_agent_config=client_config)
client_communicator = GRPCClientCommunicator(
    client_id = client_agent.get_id(),
    **client_config.comm_configs.grpc_configs,
)

[2024-04-24 14:46:18,397 INFO Client1]: Logging to ./output/result_Client1_2024-04-24 14:46:18.txt


## Start the FL experiment

Client start the FL experiment by doing the following things:

- Obtain general client-side configurations from the server and load them
- Obtain the initial global model from the server
- *[Optional]* Send the number of local data to the server
- Iteratively train the model and update the global model until receiving a `DONE` status flag from the server.

💡 The server is also logging several information regarding the recipe of client requests.

In [7]:
# Obtain general client-side configurations from the server and load them
client_config = client_communicator.get_configuration()
client_agent.load_config(client_config)

# Obtain the initial global model from the server
init_global_model = client_communicator.get_global_model(init_model=True)
client_agent.load_parameters(init_global_model)

# Send the number of local data to the server
sample_size = client_agent.get_sample_size()
client_communicator.invoke_custom_action(action='set_sample_size', sample_size=sample_size)

while True:
    client_agent.train()
    local_model = client_agent.get_parameters()
    new_global_model, metadata = client_communicator.update_global_model(local_model)
    if metadata['status'] == 'DONE':
        break
    client_agent.load_parameters(new_global_model)

[2024-04-24 14:46:31,588 INFO Client1]:      Round   Pre Val?       Time Train Loss Train Accuracy   Val Loss Val Accuracy
[2024-04-24 14:46:32,804 INFO Client1]:          0          Y                                          2.3021       8.4700
[2024-04-24 14:46:36,573 INFO Client1]:          0          N     3.7683     0.0661        79.7188     0.2003      93.7200
[2024-04-24 14:46:37,813 INFO Client1]:          1          Y                                          0.2003      93.7200
[2024-04-24 14:46:41,588 INFO Client1]:          1          N     3.7746     0.0195        94.4844     0.1124      96.4600
[2024-04-24 14:46:42,855 INFO Client1]:          2          Y                                          0.1124      96.4600
[2024-04-24 14:46:46,659 INFO Client1]:          2          N     3.8032     0.0138        95.9375     0.0996      96.8400
[2024-04-24 14:46:47,895 INFO Client1]:          3          Y                                          0.0996      96.8400
[2024-04-24 14:4